# Make the master table

In [1]:
import os 
import sys
import pandas as pd
import subprocess as sp
import glob
os.chdir('/mnt/BioHome/jreyna/jreyna/projects/dchallenge/')
hic_python='/mnt/BioHome/jreyna/software/anaconda3/envs/hic_tls/bin/python'

# default values for the command line
# sys.argv = [0] * 8
# sys.argv[1] =  'results/main/2021_Nikhil_eQTL/Results/Colocalization/T1D_34012112_Gaulton/'
# sys.argv[1] += 'BLUEPRINT_eQTL_Monocyte/FINAL_Summary_Coloc_Gene_SNP_Pairs.bed'
# sys.argv[2] = 'results/refs/ensembl/gencode.v19.annotation.bed'
# sys.argv[3] = 'results/main/2021_Nikhil_eQTL/Data/FitHiChIP_Loops/CM/FitHiChIP_L/FitHiChIP.interactions_FitHiC_Q0.01.bed'
# sys.argv[4] = 'results/refs/spp/SPP_D-Challenge_networks.xlsx'
# sys.argv[5] = 'results/refs/hg19/hg19.chrom.sizes'
# sys.argv[6] = 'results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_summ_stats/BLUEPRINT_eQTL/Monocyte.txt.gz'
# sys.argv[7] = 'results/main/loop_analysis/washU/'

# # parsing the commandline arguments
# coloc_fn = sys.argv[1]
# genes_fn = sys.argv[2]
# loop_fn = sys.argv[3]
# spp_fn = sys.argv[4]
# gs_fn = sys.argv[5]
# eqtl_fn = sys.argv[6]
# outdir = sys.argv[7]

# # setting the output file names
# os.makedirs(outdir, exist_ok=True)

In [2]:
# %load ./scripts/loop_analysis/Annotation_SNPs_with_Surround_Genes_Loops.sh
#!/usr/bin/env python

# In[1]:


# Commandline Specification
# Input
# ----------------------------------------------
# 1) colocalization file  x
# 2) gencode gene annotation file in bed format x
# 3) loop data x 
# 4) spp network data x 
# 5) reference chrom sizes x 
# Output
# ----------------------------------------------
# 1) snp-gene loop summary file
# 2) snp-gene pairs longrange file with index
# 3) snp-gene loops longrange file with index

# default values for the command line
#sys.argv = [0] * 7
#sys.argv[1] =  'results/main/2021_Nikhil_eQTL/Results/Colocalization/T1D_34012112_Gaulton/'
#sys.argv[1] += 'BLUEPRINT_eQTL_Monocyte/FINAL_Summary_Coloc_Gene_SNP_Pairs.bed'
#sys.argv[2] = 'results/refs/ensembl/gencode.v19.annotation.bed'
#sys.argv[3] = 'results/main/2021_Nikhil_eQTL/Data/FitHiChIP_Loops/CM/FitHiChIP_L/FitHiChIP.interactions_FitHiC_Q0.01.bed'
#sys.argv[4] = 'results/refs/spp/SPP_D-Challenge_networks.xlsx'
#sys.argv[5] = 'results/refs/hg19/hg19.chrom.sizes'
#sys.argv[6] = 'results/main/loop_analysis/washU/'
script = "scripts/loop_analysis/Annotation_SNPs_with_Surround_Genes_Loops.py"
gs = "results/refs/hg19/hg19.chrom.sizes"
gencode = "results/refs/ensembl/gencode.v19.annotation.bed"
spp = "results/refs/spp/SPP_D-Challenge_networks.xlsx"

# template file names for colocalization, hichip, eqtl and outdir
coloc_tmp = "results/main/2021_Nikhil_eQTL/Results/Colocalization/{gwas}/{eqtl}_{cline}/FINAL_Summary_Coloc_Gene_SNP_Pairs.bed"
fithichip_tmp = "results/main/2021_Nikhil_eQTL/Data/FitHiChIP_Loops/{cline}/FitHiChIP_S/FitHiChIP.interactions_FitHiC_Q0.01.bed"
eqtl_tmp = "results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_summ_stats/{eqtl}/{cline}.txt.gz"
outdir_tmp = "results/main/loop_analysis/{gwas}/{dice_cline}/{eqtl_study}/{eqtl_cline}/"

# getting a list of DICE clines with HiChIP data
fithichip_clines = glob.glob("results/main/2021_Nikhil_eQTL/Data/FitHiChIP_Loops/*")
fithichip_clines = [x.split('/')[-1] for x in fithichip_clines]

# eqtl mapper
eqtl_mapper = pd.read_table('results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_summ_stats/eqtl.cline.mapper.study.tsv')
eqtl_mapper.sort_values(['will_analyze', 'study', 'dice_shortname'], ascending=[False, True, True], inplace=True)
eqtl_mapper[['study', 'dice_shortname', 'will_analyze', 'eqtl_file']]
eqtl_mapper = eqtl_mapper.loc[eqtl_mapper['will_analyze'] == 'Yes']
eqtl_mapper = eqtl_mapper.loc[eqtl_mapper.dice_shortname.isin(fithichip_clines)]
eqtl_clines = eqtl_mapper['eqtl_file'].str.split('/')
eqtl_clines = [x[-1].split('.')[0] for x in eqtl_clines]
eqtl_mapper['eqtl_cline'] = eqtl_clines
eqtl_mapper = eqtl_mapper[['study', 'dice_shortname', 'eqtl_cline', 'eqtl_file']]

In [3]:
eqtl_mapper.head()

,study,dice_shortname,eqtl_cline,eqtl_file
0,BLUEPRINT_eQTL,CM,Monocyte,results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_s...
1,BLUEPRINT_eQTL,NCM,Monocyte,results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_s...
2,BLUEPRINT_eQTL,TFH,T-cell,results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_s...
3,BLUEPRINT_eQTL,TH1,T-cell,results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_s...
4,BLUEPRINT_eQTL,TH17,T-cell,results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_s...


In [4]:
# get gwas runs
gwas_runs = [os.path.basename(x) for x in glob.glob('results/main/2021_Nikhil_eQTL/Data/T1D_GWAS/*')]
gwas_runs = gwas_runs[2:3]

# get eqtl runs 
eqtl_runs = [x for x in glob.glob('results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_summ_stats/*/*')]
eqtl_runs = [x for x in eqtl_runs if x != 'README' and 'eQTL' in x]

In [5]:
eqtl_mapper.head()

,study,dice_shortname,eqtl_cline,eqtl_file
0,BLUEPRINT_eQTL,CM,Monocyte,results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_s...
1,BLUEPRINT_eQTL,NCM,Monocyte,results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_s...
2,BLUEPRINT_eQTL,TFH,T-cell,results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_s...
3,BLUEPRINT_eQTL,TH1,T-cell,results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_s...
4,BLUEPRINT_eQTL,TH17,T-cell,results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_s...


In [32]:
for gwas in gwas_runs:
    for i, sr in eqtl_mapper.iterrows():
        
        coloc = coloc_tmp.format(gwas=gwas, eqtl=sr.study, cline=sr.eqtl_cline)
        fithichip = fithichip_tmp.format(cline=sr.dice_shortname)
        outdir = outdir_tmp.format(gwas=gwas, 
                                   dice_cline=sr.dice_shortname,
                                   eqtl_cline=sr.eqtl_cline,
                                   eqtl_study=sr.study)
        cmd = "{} {} {} {} {} {} {} {} {}".format(hic_python, script, coloc, gencode, 
                                                  fithichip, spp, gs, sr.eqtl_file, outdir)        

        
        if not os.path.exists(outdir):
            job = sp.Popen(cmd, shell=True, stderr=sp.PIPE, stdout=sp.PIPE)
            out, err = job.communicate()
            print('Now running: {}'.format(cmd))
            print(out.decode())
            print(err.decode())
            print('Results can now be found at: {}'.format(outdir))

        else:
            msg = 'Previously ran {}'.format(outdir)
            print(msg)
            
        print()

Previously ran results/main/loop_analysis/T1D_34012112_Gaulton/CM/BLUEPRINT_eQTL/Monocyte/

Previously ran results/main/loop_analysis/T1D_34012112_Gaulton/NCM/BLUEPRINT_eQTL/Monocyte/

Previously ran results/main/loop_analysis/T1D_34012112_Gaulton/TFH/BLUEPRINT_eQTL/T-cell/

Previously ran results/main/loop_analysis/T1D_34012112_Gaulton/TH1/BLUEPRINT_eQTL/T-cell/

Previously ran results/main/loop_analysis/T1D_34012112_Gaulton/TH17/BLUEPRINT_eQTL/T-cell/

Previously ran results/main/loop_analysis/T1D_34012112_Gaulton/TH2/BLUEPRINT_eQTL/T-cell/

Previously ran results/main/loop_analysis/T1D_34012112_Gaulton/THSTAR/BLUEPRINT_eQTL/T-cell/

Previously ran results/main/loop_analysis/T1D_34012112_Gaulton/TREGMEM/BLUEPRINT_eQTL/T-cell/

Previously ran results/main/loop_analysis/T1D_34012112_Gaulton/TREGNAIVE/BLUEPRINT_eQTL/T-cell/

Previously ran results/main/loop_analysis/T1D_34012112_Gaulton/CD4N/DICE_eQTL/CD4_NAIVE/

Previously ran results/main/loop_analysis/T1D_34012112_Gaulton/CD4N/DICE_e